# 4/23 - Barents Sea example

In this notebook we want to attempt a few things in the Barents Sea as an example:

- TS and volume distributions over 2014
- TS and volume distributinos over the 5-year period
- specific properties and their variation with T and S (T will be on x axis, heat flux sum on y-axis as an example, or summed advective flux on y-axis)
- label water masses or think about what can be seen in these TS diagrams

### Import packages, libraries, functions

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import xarray as xr
import h5py
from scipy.io import loadmat
import matplotlib as mpl
import time
import gsw

# import existing python files
plt.rcParams['figure.figsize'] = (10,4)

# add rdmds reading functions to path
sys.path.append("/home/mmurakami/MITgcm/MITgcm_c68r/MITgcm-checkpoint68r/utils/python/MITgcmutils/MITgcmutils/") # go to parent dir
from mds import *

# add the other files
sys.path.append("/home/mmurakami/crios_backups/an_helper_functions")
from read_binary import *
from aste_helper_funcs import *
from ts2dte import ts2dte

1992-01-14 21:20:00


### Get the timesteps we want from the timestep.txt file

In [2]:
inf_name = "/scratch/atnguyen/aste_270x450x180/OFFICIAL_ASTE_R1_Sep2019/timestep.txt"
with open(inf_name) as inf:
    lines = inf.readlines()
inf.close()

timestep = np.zeros((len(lines),len(lines[0].split())))
for line in range(len(lines)):
    myline = lines[line].split()
    timestep[line] = myline

timestep = timestep.astype(int)   # this is now the list of files, index

In [5]:
# we want two subsets of data 2014 and 2006-2011
p1 =np.array([2014])
p2 = np.arange(2006,2012,1)

In [6]:
r1 = timestep[np.isin(timestep[:,0],p1)]

In [28]:
r2 = timestep[np.isin(timestep[:,0],p2)]
r2[:5]

array([[  2006,      1,      1, 210384, 732678],
       [  2006,      2,      1, 214848, 732709],
       [  2006,      3,      1, 218880, 732737],
       [  2006,      4,      1, 223344, 732768],
       [  2006,      5,      1, 227664, 732798]])

In [29]:
# see how this compares to the function
# we have these in a file but let's try to plot them from the function
ts = 4464
dt = 0
startyr = 2006
dte = ts2dte(ts,deltat=dt,startyr=startyr)  # should be default to start from january 1
print(dte.strftime("%Y-%m-%d %H:%M:%S"))

2006-01-01 00:00:00


In [30]:
# see how this compares to the function
# we have these in a file but let's try to plot them from the function
ts = 4464
dt = 600
startyr = 2006
dte = ts2dte(ts,deltat=dt,startyr=startyr)  # should be default to start from january 1
print(dte.strftime("%Y-%m-%d %H:%M:%S"))

2006-02-01 00:00:00


In [5]:
# in data file (ie how we get to days from the labeling of the files:
'''
2002.01.01 to 2017.12.31 + 1 day: 366*4+365*12=5844days*24*3600/600 + 1ts
nTimeSteps=841537,
'''
841537-4464

837073

In [2]:
4464*600/3600/24

31.0

In [3]:
4472*600/3600/24

31.055555555555557

In [4]:
4032*600/3600/24

28.0

In [ ]:
# this is h